In [ ]:
!pip install tinymlgen

In [ ]:
import math
import numpy as np
from sklearn.datasets import load_digits
import tensorflow as tf
from tensorflow.keras import layers
from tinymlgen import port

In [ ]:
from tensorflow.keras.datasets import mnist
import cv2

In [ ]:
np.random.seed(1337)
(x_values, y_values), (x_test, y_test) = mnist.load_data()
x_values = x_values / x_values.max()
batch_tensor = tf.reshape(x_values, [60000, 28, 28, 1])
resized_images = tf.image.resize(batch_tensor, [14,14])

In [ ]:
resized_images=resized_images.numpy()
resized_images.max()

In [ ]:
TRAIN_SPLIT = int(0.6 * len(resized_images))
TEST_SPLIT = int(0.2 * len(resized_images) + TRAIN_SPLIT)
x_train, x_test, x_validate = np.split(resized_images, [TRAIN_SPLIT, TEST_SPLIT])
y_train, y_test, y_validate = np.split(y_values, [TRAIN_SPLIT, TEST_SPLIT])

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Conv2D(4, (3, 3), activation='relu', padding='same', input_shape=(14, 14, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(len(np.unique(y_train))))
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(x_train, y_train, epochs=12, batch_size=128, validation_data=(x_validate, y_validate))

In [ ]:
x_test = (x_test / x_test.max()).reshape((len(x_test), 14, 14, 1))
y_pred = model.predict(x_test).argmax(axis=1)
print('ACCURACY', (y_pred == y_test).sum() / len(y_test))

In [ ]:
c_code = port(model, optimize=False, variable_name='digits_model', pretty_print=True)
print(c_code)

In [ ]:
from google.colab import files
import os
os.chdir('/content/sample_data/')
open('output.h','w').write(c_code)

In [ ]:
model.save("mnist_ard.h5")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)